#### Import Libraries and Change Directory

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import xlrd
import statsmodels.formula.api as smf

In [2]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Yearly_Player_Rankings')
os.getcwd()

'C:\\Users\\marcd\\Python_Stuff\\fantasy_football\\Yearly_Player_Rankings'

#### Load Yearly Player Data

In [3]:
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-4:] == 'xlsx']
yearly = pd.DataFrame()
for f in files_csv:
    data = pd.read_excel(f)
    yearly = yearly.append(data, ignore_index=True)
yearly.fillna(0, inplace=True)
yearly

,Rk,Year,Player,Tm,FantPos,Age,G,Pass Cmp,Pass Att,Pass Yds,...,Rec Tgt,Rec,Rec Yds,Rec Y/R,Rec TD,Fmb,FL,TD,2PM,2PP
0,1,2019,Christian McCaffrey,CAR,RB,23,16,0,2,0,...,142,116,1005,8.66,4,1.0,0,19,1.0,0.0
1,2,2019,Lamar Jackson,BAL,QB,22,15,265,401,3127,...,0,0,0,0.00,0,9.0,2,7,0.0,0.0
2,3,2019,Derrick Henry,TEN,RB,25,15,0,0,0,...,24,18,206,11.44,2,5.0,3,18,0.0,0.0
3,4,2019,Aaron Jones,GNB,RB,25,16,0,0,0,...,68,49,474,9.67,3,3.0,2,19,0.0,0.0
4,5,2019,Ezekiel Elliott,DAL,RB,24,16,0,0,0,...,71,54,420,7.78,2,3.0,2,14,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492,495,2021,Reggie Gilliam,BUF,RB,24,16,0,0,0,...,5,3,23,7.67,0,0.0,0,0,0.0,0.0
1493,496,2021,Chad Henne,KAN,QB,36,4,11,16,82,...,0,0,0,0.00,0,2.0,0,0,0.0,0.0
1494,497,2021,K.J. Hill,LAC,WR,24,6,0,0,0,...,3,3,30,10.00,0,0.0,0,0,0.0,0.0
1495,498,2021,Kylin Hill,GNB,RB,23,8,0,0,0,...,2,1,5,5.00,0,0.0,0,0,0.0,0.0


#### Scoring Points System Modifier

In [4]:
RR_TD = 6     # Rushing/Receiving TDs
RR_YDS = 0.1  # Rushing/Receiving Yards
REC = 1     # Receptions
FL = -2       # Fumble Loses
P_TD = 4      # Passing TDs
P_YDS = 0.04  # Passing Yards
INT = -2      # Interceptions
X_CONV = 2    # 2-point Conversions

yearly['FPTS'] = RR_TD*(yearly['Rush TD'] + yearly['Rec TD']) + RR_YDS*(yearly['Rush Yds'] + yearly['Rec Yds']) + REC*(yearly['Rec']) + \
FL*(yearly['FL']) + P_TD*(yearly['Pass TD']) + P_YDS*(yearly['Pass Yds']) + INT*(yearly['Pass Int']) + X_CONV*(yearly['2PM'] + yearly['2PP'])

yearly['FPTS/G'] = yearly['FPTS'] / yearly['G']

yearly = yearly.round({'FPTS/G': 2})
yearly

,Rk,Year,Player,Tm,FantPos,Age,G,Pass Cmp,Pass Att,Pass Yds,...,Rec Yds,Rec Y/R,Rec TD,Fmb,FL,TD,2PM,2PP,FPTS,FPTS/G
0,1,2019,Christian McCaffrey,CAR,RB,23,16,0,2,0,...,1005,8.66,4,1.0,0,19,1.0,0.0,471.20,29.45
1,2,2019,Lamar Jackson,BAL,QB,22,15,265,401,3127,...,0,0.00,0,9.0,2,7,0.0,0.0,415.68,27.71
2,3,2019,Derrick Henry,TEN,RB,25,15,0,0,0,...,206,11.44,2,5.0,3,18,0.0,0.0,294.60,19.64
3,4,2019,Aaron Jones,GNB,RB,25,16,0,0,0,...,474,9.67,3,3.0,2,19,0.0,0.0,314.80,19.68
4,5,2019,Ezekiel Elliott,DAL,RB,24,16,0,0,0,...,420,7.78,2,3.0,2,14,0.0,0.0,311.70,19.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492,495,2021,Reggie Gilliam,BUF,RB,24,16,0,0,0,...,23,7.67,0,0.0,0,0,0.0,0.0,5.60,0.35
1493,496,2021,Chad Henne,KAN,QB,36,4,11,16,82,...,0,0.00,0,2.0,0,0,0.0,0.0,3.28,0.82
1494,497,2021,K.J. Hill,LAC,WR,24,6,0,0,0,...,30,10.00,0,0.0,0,0,0.0,0.0,6.00,1.00
1495,498,2021,Kylin Hill,GNB,RB,23,8,0,0,0,...,5,5.00,0,0.0,0,0,0.0,0.0,3.90,0.49


#### Save Dataframe as an Excel Sheet

In [5]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football')
writer = pd.ExcelWriter('yearly_relevant.xlsx')
yearly.to_excel(writer,'sheet1')
writer.save()

In [ ]:
def get_QB_data(dir, df, year):
    os.chdir(dir)
    files = os.listdir(path)
    files_csv = [f for f in files if f[-3:] == 'csv']
    df = pd.DataFrame()
    WEEK = 1
    for f in files_csv:
        data = pd.read_csv(f)
        data['WEEK'] = WEEK
        df = df.append(data, ignore_index=True)
        WEEK += 1
    df.fillna(0, inplace=True)
    df = df.drop(columns=['Y/A','SACKS','PCT','G','FPTS','FPTS/G','ROST'])
    df = df.rename(columns={'CMP':'PASS_CMP','ATT':'PASS_ATT','YDS':'PASS_YDS','TD':'PASS_TD','INT':'PASS_INT','ATT.1':'RUSH_ATT','YDS.1':'RUSH_YDS','TD.1':'RUSH_TD','Rank':'RANK','Player':'PLAYER'})
    df['YEAR'] = year
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[-2:-1] + cols[:11]
    df = df[cols]
    pd.set_option('precision', 0)
    df[['PLAYER','TEAM']] = df['PLAYER'].str.split('(', expand=True)
    df['TEAM'] = df['TEAM'].str.replace(r')','')
    cols = df.columns.tolist()
    cols = cols[:4] + cols[-1:] + cols[4:13]
    df = df[cols]
    df = df[df['PLAYER'].notna()]

In [ ]:
dir = 'C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2019/QB'
df = 'QB_2019'
year = 2019
get_QB_data(dir, df, year)

#### Get 2019 Weekly Data

In [2]:
#QB - 2019
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2019/QB')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
QB_2019 = pd.DataFrame()
for f in files_csv:
    data = pd.read_csv(f)
    WEEK = f[:-4]
    WEEK = WEEK[12:]
    WEEK = int(WEEK)
    data['WEEK'] = WEEK
    QB_2019 = QB_2019.append(data, ignore_index=True)
QB_2019.fillna(0, inplace=True)
QB_2019 = QB_2019.drop(columns=['Y/A','SACKS','PCT','FPTS','FPTS/G','ROST'])
QB_2019 = QB_2019.rename(columns={'CMP':'PASS_CMP','ATT':'PASS_ATT','YDS':'PASS_YDS','TD':'PASS_TD','INT':'PASS_INT','ATT.1':'RUSH_ATT','YDS.1':'RUSH_YDS','TD.1':'RUSH_TD','Rank':'RANK','Player':'PLAYER'})
QB_2019['YEAR'] = 2019
pd.set_option('precision', 0)
QB_2019[['PLAYER','TEAM']] = QB_2019['PLAYER'].str.split('(', expand=True)
QB_2019['TEAM'] = QB_2019['TEAM'].str.replace(r')','')
QB_2019['POSITION'] = 'QB'
cols = QB_2019.columns.tolist()
cols = cols[13:14] + cols[12:13] + cols[0:2] + cols[-1:] + cols[-2:-1] + cols[-5:-4] + cols[2:11]
QB_2019 = QB_2019[cols]
QB_2019 = QB_2019[QB_2019['PLAYER'].notna()]
QB_2019 = QB_2019[QB_2019['G'] == 1]
QB_2019

C:\Users\marcd\AppData\Local\Temp/ipykernel_20332/4220308504.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  QB_2019['TEAM'] = QB_2019['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,G,PASS_CMP,PASS_ATT,PASS_YDS,PASS_TD,PASS_INT,RUSH_ATT,RUSH_YDS,RUSH_TD,FL
0,2019,1,1,Lamar Jackson,QB,BAL,1,17,20,324,5,0,3,6,0,0
1,2019,1,2,Dak Prescott,QB,DAL,1,25,32,405,4,0,4,12,0,0
2,2019,1,3,Deshaun Watson,QB,CLE,1,20,30,268,3,1,4,40,1,0
3,2019,1,4,Matthew Stafford,QB,LAR,1,27,45,385,3,0,3,22,0,1
4,2019,1,5,Patrick Mahomes II,QB,KC,1,25,33,378,3,0,1,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,2019,9,25,Mason Rudolph,QB,PIT,1,26,35,191,1,1,3,5,0,0
2201,2019,9,26,Gardner Minshew II,QB,PHI,1,27,47,309,0,2,4,34,0,2
2202,2019,9,27,Dwayne Haskins,QB,FA,1,15,22,144,0,0,3,14,0,0
2203,2019,9,28,Mitchell Trubisky,QB,PIT,1,10,21,125,0,0,2,15,0,0


In [242]:
#RB - 2019
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2019/RB')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
RB_2019 = pd.DataFrame()
for f in files_csv:
    data = pd.read_csv(f)
    WEEK = f[:-4]
    WEEK = WEEK[12:]
    WEEK = int(WEEK)
    data['WEEK'] = WEEK
    data['WEEK'] = WEEK
    RB_2019 = RB_2019.append(data, ignore_index=True)
RB_2019.fillna(0, inplace=True)
RB_2019 = RB_2019.drop(columns=['Y/A','LG','20+','Y/R','FPTS','FPTS/G','ROST'])
RB_2019 = RB_2019.rename(columns={'ATT':'RUSH_ATT','YDS':'RUSH_YDS','TD':'RUSH_TD','YDS.1':'REC_YDS','TD.1':'REC_TD','Rank':'RANK','Player':'PLAYER'})
RB_2019['YEAR'] = 2019
pd.set_option('precision', 0)
RB_2019[['PLAYER','TEAM']] = RB_2019['PLAYER'].str.split('(', expand=True)
RB_2019['TEAM'] = RB_2019['TEAM'].str.replace(r')','')
RB_2019['POSITION'] = 'RB'
cols = RB_2019.columns.tolist()
cols = cols[12:13] + cols[11:12] + cols[:2] + cols[-1:] + cols[-2:-1] + cols[-5:-4] + cols[2:10]
RB_2019 = RB_2019[cols]
RB_2019 = RB_2019[RB_2019['PLAYER'].notna()]
RB_2019 = RB_2019[RB_2019['G'] == 1]
RB_2019

C:\Users\marcd\AppData\Local\Temp/ipykernel_23064/4044828667.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  RB_2019['TEAM'] = RB_2019['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,G,RUSH_ATT,RUSH_YDS,RUSH_TD,REC,TGT,REC_YDS,REC_TD,FL
0,2019,1,1,Christian McCaffrey,RB,CAR,1,19,128,2,10,11,81,0,0
1,2019,1,2,Austin Ekeler,RB,LAC,1,12,58,1,6,7,96,2,0
2,2019,1,3,Derrick Henry,RB,TEN,1,19,84,1,1,2,75,1,0
3,2019,1,4,Marlon Mack,RB,HOU,1,25,174,1,0,0,0,0,0
4,2019,1,5,Dalvin Cook,RB,MIN,1,21,111,2,2,2,9,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4997,2019,9,284,Michael Burton,RB,KC,1,0,0,0,0,0,0,0,0
4998,2019,9,285,Senorise Perry,RB,FA,1,0,0,0,0,0,0,0,0
5000,2019,9,287,Roosevelt Nix,RB,FA,1,0,0,0,0,1,0,0,0
5002,2019,9,289,Raheem Mostert,RB,MIA,1,1,-2,0,0,0,0,0,0


In [243]:
#WR - 2019
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2019/WR')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
WR_2019 = pd.DataFrame()
for f in files_csv:
    data = pd.read_csv(f)
    WEEK = f[:-4]
    WEEK = WEEK[12:]
    WEEK = int(WEEK)
    data['WEEK'] = WEEK
    data['WEEK'] = WEEK
    WR_2019 = WR_2019.append(data, ignore_index=True)
WR_2019.fillna(0, inplace=True)
WR_2019 = WR_2019.drop(columns=['LG','20+','Y/R','FPTS','FPTS/G','ROST'])
WR_2019 = WR_2019.rename(columns={'YDS':'REC_YDS','TD':'REC_TD','ATT':'RUSH_ATT','YDS.1':'RUSH_YDS','TD.1':'RUSH_TD','Rank':'RANK','Player':'PLAYER'})
WR_2019['YEAR'] = 2019
pd.set_option('precision', 0)
WR_2019[['PLAYER','TEAM']] = WR_2019['PLAYER'].str.split('(', expand=True)
WR_2019['TEAM'] = WR_2019['TEAM'].str.replace(r')','')
WR_2019['POSITION'] = 'WR'
cols = WR_2019.columns.tolist()
cols = cols[12:13] + cols[11:12] + cols[:2] + cols[-1:] + cols[-2:-1] + cols[-5:-4] + cols[2:10]
WR_2019 = WR_2019[cols]
WR_2019 = WR_2019[WR_2019['PLAYER'].notna()]
WR_2019 = WR_2019[WR_2019['G'] == 1]
WR_2019

C:\Users\marcd\AppData\Local\Temp/ipykernel_23064/1274792873.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  WR_2019['TEAM'] = WR_2019['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,G,REC,TGT,REC_YDS,REC_TD,RUSH_ATT,RUSH_YDS,RUSH_TD,FL
0,2019,1,1,Sammy Watkins,WR,GB,1,9,11,198,3,0,0,0,0
1,2019,1,2,DeSean Jackson,WR,FA,1,8,9,154,2,0,0,0,0
2,2019,1,3,John Ross,WR,FA,1,7,12,158,2,0,0,0,0
3,2019,1,4,Marquise Brown,WR,ARI,1,4,5,147,2,0,0,0,0
4,2019,1,5,DeAndre Hopkins,WR,ARI,1,8,13,111,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7952,2019,9,450,Jake Kumerow,WR,FA,1,0,2,0,0,0,0,0,0
7953,2019,9,451,Ty Montgomery,WR,NE,1,0,0,0,0,0,0,0,0
7955,2019,9,453,Jarius Wright,WR,FA,1,0,4,0,0,1,-7,0,0
7956,2019,9,454,Mecole Hardman,WR,KC,1,0,0,0,0,1,7,0,1


In [244]:
#TE - 2019
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2019/TE')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
TE_2019 = pd.DataFrame()
for f in files_csv:
    data = pd.read_csv(f)
    WEEK = f[:-4]
    WEEK = WEEK[12:]
    WEEK = int(WEEK)
    data['WEEK'] = WEEK
    data['WEEK'] = WEEK
    TE_2019 = TE_2019.append(data, ignore_index=True)
TE_2019.fillna(0, inplace=True)
TE_2019 = TE_2019.drop(columns=['LG','20+','Y/R','FPTS','FPTS/G','ROST'])
TE_2019 = TE_2019.rename(columns={'YDS':'REC_YDS','TD':'REC_TD','ATT':'RUSH_ATT','YDS.1':'RUSH_YDS','TD.1':'RUSH_TD','Rank':'RANK','Player':'PLAYER'})
TE_2019['YEAR'] = 2019
pd.set_option('precision', 0)
TE_2019[['PLAYER','TEAM']] = TE_2019['PLAYER'].str.split('(', expand=True)
TE_2019['TEAM'] = TE_2019['TEAM'].str.replace(r')','')
TE_2019['POSITION'] = 'TE'
cols = TE_2019.columns.tolist()
cols = cols[12:13] + cols[11:12] + cols[:2] + cols[-1:] + cols[-2:-1] + cols[-5:-4] + cols[2:10]
TE_2019 = TE_2019[cols]
TE_2019 = TE_2019[TE_2019['PLAYER'].notna()]
TE_2019 = TE_2019[TE_2019['G'] == 1]
TE_2019

C:\Users\marcd\AppData\Local\Temp/ipykernel_23064/740609330.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  TE_2019['TEAM'] = TE_2019['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,G,REC,TGT,REC_YDS,REC_TD,RUSH_ATT,RUSH_YDS,RUSH_TD,FL
0,2019,1,1,Evan Engram,TE,JAC,1,11,14,116,1,0,0,0,0
1,2019,1,2,T.J. Hockenson,TE,DET,1,6,9,131,1,0,0,0,0
2,2019,1,3,Mark Andrews,TE,BAL,1,8,8,108,1,0,0,0,0
3,2019,1,4,Delanie Walker,TE,FA,1,5,6,55,2,0,0,0,0
4,2019,1,5,Vernon Davis,TE,FA,1,4,7,59,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4665,2019,9,262,Ben Koyack,TE,FA,1,0,1,0,0,0,0,0,0
4666,2019,9,263,Tyler Kroft,TE,SF,1,0,1,0,0,0,0,0,0
4667,2019,9,264,MyCole Pruitt,TE,FA,1,0,0,0,0,0,0,0,0
4668,2019,9,265,Chris Manhertz,TE,JAC,1,0,0,0,0,0,0,0,0


#### Merge All 2019 Weekly Data Together

In [245]:
weekly_2019 = QB_2019.append([RB_2019, WR_2019, TE_2019])
weekly_2019.fillna(0, inplace=True)

RR_TD = 6     # Rushing/Receiving TDs
RR_YDS = 0.1  # Rushing/Receiving Yards
REC = 1       # Receptions
FL = -2       # Fumble Loses
P_TD = 4      # Passing TDs
P_YDS = 0.04  # Passing Yards
INT = -2      # Interceptions

weekly_2019['FPTS'] = RR_TD*(weekly_2019['RUSH_TD'] + weekly_2019['REC_TD']) + RR_YDS*(weekly_2019['RUSH_YDS'] + weekly_2019['REC_YDS']) + REC*(weekly_2019['REC']) + \
FL*(weekly_2019['FL']) + P_TD*(weekly_2019['PASS_TD']) + P_YDS*(weekly_2019['PASS_YDS']) + INT*(weekly_2019['PASS_INT'])

weekly_2019 = weekly_2019.round({'FPTS': 2})
weekly_2019

weekly_2019

,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,G,PASS_CMP,PASS_ATT,PASS_YDS,...,PASS_INT,RUSH_ATT,RUSH_YDS,RUSH_TD,FL,REC,TGT,REC_YDS,REC_TD,FPTS
0,2019,1,1,Lamar Jackson,QB,BAL,1,17,20,324,...,0,3,6,0,0,0,0,0,0,34
1,2019,1,2,Dak Prescott,QB,DAL,1,25,32,405,...,0,4,12,0,0,0,0,0,0,33
2,2019,1,3,Deshaun Watson,QB,CLE,1,20,30,268,...,1,4,40,1,0,0,0,0,0,31
3,2019,1,4,Matthew Stafford,QB,LAR,1,27,45,385,...,0,3,22,0,1,0,0,0,0,28
4,2019,1,5,Patrick Mahomes II,QB,KC,1,25,33,378,...,0,1,2,0,0,0,0,0,0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4665,2019,9,262,Ben Koyack,TE,FA,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4666,2019,9,263,Tyler Kroft,TE,SF,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4667,2019,9,264,MyCole Pruitt,TE,FA,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4668,2019,9,265,Chris Manhertz,TE,JAC,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Add 2019 Schedule / Opponent column

In [246]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Schedule')
schedule_2019 = pd.read_csv('2019_Schedule.csv')
schedule_2019 = schedule_2019.drop(['W18'], axis=1)
schedule_2019 = schedule_2019.rename(columns={'Team':'TEAM'})
schedule_2019 = schedule_2019.melt(id_vars=['TEAM'], var_name='WEEK')
schedule_2019['WEEK'] = schedule_2019['WEEK'].map(lambda x: x.lstrip('W'))
schedule_2019['value'] = schedule_2019['value'].map(lambda x: x.lstrip('@'))
schedule_2019 = schedule_2019.astype({'WEEK': int})
schedule_2019

,TEAM,WEEK,value
0,ARI,1,DET
1,ATL,1,MIN
2,BAL,1,MIA
3,BUF,1,NYJ
4,CAR,1,LAR
...,...,...,...
539,SEA,17,SF
540,SF,17,SEA
541,TB,17,ATL
542,TEN,17,HOU


In [247]:
weekly_2019 = pd.merge(weekly_2019, schedule_2019, on=['TEAM','WEEK'])
weekly_2019 = weekly_2019.rename(columns={'value':'OPP'})
cols = weekly_2019.columns.tolist()
cols = cols[0:6] + cols[-1:] + cols[6:15] + cols[16:20] + cols[15:16] + cols[20:21]
weekly_2019 = weekly_2019[cols]
weekly_2019

,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,OPP,G,PASS_CMP,PASS_ATT,...,PASS_INT,RUSH_ATT,RUSH_YDS,RUSH_TD,REC,TGT,REC_YDS,REC_TD,FL,FPTS
0,2019,1,1,Lamar Jackson,QB,BAL,MIA,1,17,20,...,0,3,6,0,0,0,0,0,0,34
1,2019,1,37,Mike Davis,RB,BAL,MIA,1,0,0,...,0,5,19,0,6,7,17,0,0,10
2,2019,1,42,Gus Edwards,RB,BAL,MIA,1,0,0,...,0,17,56,0,0,0,0,0,0,6
3,2019,1,59,Justice Hill,RB,BAL,MIA,1,0,0,...,0,7,27,0,0,0,0,0,0,3
4,2019,1,296,Patrick Ricard,RB,BAL,MIA,1,0,0,...,0,0,0,0,1,1,1,1,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4466,2019,6,65,Byron Pringle,WR,CHI,BYE,1,0,0,...,0,0,0,0,2,4,24,0,0,4
4467,2019,6,450,Tajae Sharpe,WR,CHI,BYE,1,0,0,...,0,0,0,0,0,2,0,0,0,0
4468,2019,6,9,Ryan Griffin,TE,CHI,BYE,1,0,0,...,0,0,0,0,3,3,28,1,0,12
4469,2019,8,83,James Washington,WR,DAL,BYE,1,0,0,...,0,0,0,0,1,6,21,0,0,3


#### Add Opponent Position Defensive Rankings and Remove Irrelevant Players and Add FPTS/G

In [248]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Defense_Rankings')
defense_2019 = pd.read_csv('Defense_Rankings_2019.csv')
defense_2019 = defense_2019.melt(id_vars=['OPP'], var_name='RANK')
defense_2019 = defense_2019.rename(columns={'RANK':'POSITION','value':'RANK'})

weekly_2019 = pd.merge(weekly_2019, defense_2019, on=['OPP','POSITION'])
G_SUM_2019 = weekly_2019.groupby('PLAYER')['G'].sum()
weekly_2019 = pd.merge(weekly_2019, G_SUM_2019, on='PLAYER')
weekly_2019 = weekly_2019.rename(columns={'RANK_x':'RANK','RANK_y':'OPP_DEF_POS_RANK','G_y':'G_SUM'})
weekly_2019 = weekly_2019.drop(['G_x'], axis=1)
cols = weekly_2019.columns.tolist()
cols = cols[0:7] + cols[-2:-1] + cols[-1:] + cols[7:21]
weekly_2019 = weekly_2019[cols]

weekly_2019 = weekly_2019.astype({'PASS_CMP':int,'PASS_ATT':int,'PASS_YDS':int,'PASS_TD':int,'PASS_INT':int,'RUSH_ATT':int,'RUSH_YDS':int,'RUSH_TD':int,'FL':int,'REC':int,'TGT':int,'REC_YDS':int,'REC_TD':int,'FPTS':float})

weekly_2019['FPTS/G'] = weekly_2019.groupby(['PLAYER'])['FPTS'].transform('mean')
weekly_2019['FPTS/G'] = weekly_2019['FPTS/G'].apply(lambda x: round(x, 2))

weekly_2019['FPTS_DIFF'] = weekly_2019['FPTS'] - weekly_2019['FPTS/G']
weekly_2019['FPTS_DIFF'] = weekly_2019['FPTS_DIFF'].apply(lambda x: round(x, 2))

weekly_2019['PLAYER'] = weekly_2019['PLAYER'].map(lambda x: x.rstrip(' '))

weekly_2019

,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,OPP,OPP_DEF_POS_RANK,G_SUM,PASS_CMP,...,RUSH_YDS,RUSH_TD,REC,TGT,REC_YDS,REC_TD,FL,FPTS,FPTS/G,FPTS_DIFF
0,2019,1,1,Lamar Jackson,QB,BAL,MIA,31,15,17,...,6,0,0,0,0,0,0,34,28,6
1,2019,15,1,Lamar Jackson,QB,BAL,NYJ,17,15,15,...,86,0,0,0,0,0,0,37,28,9
2,2019,13,11,Lamar Jackson,QB,BAL,SF,10,15,14,...,101,1,0,0,0,0,1,22,28,-5
3,2019,11,2,Lamar Jackson,QB,BAL,HOU,26,15,17,...,79,0,0,0,0,0,0,33,28,5
4,2019,14,11,Lamar Jackson,QB,BAL,BUF,3,15,16,...,40,0,0,0,0,0,0,20,28,-8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4326,2019,17,254,Trenton Irwin,WR,CIN,CLE,8,1,0,...,0,0,0,0,0,0,0,0,0,0
4327,2019,5,244,Stephen Anderson,TE,ARI,CIN,12,1,0,...,0,0,0,0,0,0,0,0,0,0
4328,2019,1,380,Equanimeous St. Brown,WR,CHI,GB,9,1,0,...,0,0,0,0,0,0,0,0,0,0
4329,2019,4,202,Tom Kennedy,WR,DET,KC,2,1,0,...,0,0,0,0,0,0,0,0,0,0


#### Save 2019 Weekly Data

In [249]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football')
writer = pd.ExcelWriter('weekly_2019.xlsx')
weekly_2019.to_excel(writer,'sheet1')
writer.save()

#### Get 2020 Weekly Data

In [250]:
#QB - 2020
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2020/QB')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
QB_2020 = pd.DataFrame()
for f in files_csv:
    data = pd.read_csv(f)
    WEEK = f[:-4]
    WEEK = WEEK[12:]
    WEEK = int(WEEK)
    data['WEEK'] = WEEK
    data['WEEK'] = WEEK
    QB_2020 = QB_2020.append(data, ignore_index=True)
QB_2020.fillna(0, inplace=True)
QB_2020 = QB_2020.drop(columns=['Y/A','SACKS','PCT','FPTS','FPTS/G','ROST'])
QB_2020 = QB_2020.rename(columns={'CMP':'PASS_CMP','ATT':'PASS_ATT','YDS':'PASS_YDS','TD':'PASS_TD','INT':'PASS_INT','ATT.1':'RUSH_ATT','YDS.1':'RUSH_YDS','TD.1':'RUSH_TD','Rank':'RANK','Player':'PLAYER'})
QB_2020['YEAR'] = 2020
pd.set_option('precision', 0)
QB_2020[['PLAYER','TEAM']] = QB_2020['PLAYER'].str.split('(', expand=True)
QB_2020['TEAM'] = QB_2020['TEAM'].str.replace(r')','')
QB_2020['POSITION'] = 'QB'
cols = QB_2020.columns.tolist()
cols = cols[13:14] + cols[12:13] + cols[0:2] + cols[-1:] + cols[-2:-1] + cols[-5:-4] + cols[2:11]
QB_2020 = QB_2020[cols]
QB_2020 = QB_2020[QB_2020['PLAYER'].notna()]
QB_2020 = QB_2020[QB_2020['G'] == 1]
QB_2020

C:\Users\marcd\AppData\Local\Temp/ipykernel_23064/3679369503.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  QB_2020['TEAM'] = QB_2020['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,G,PASS_CMP,PASS_ATT,PASS_YDS,PASS_TD,PASS_INT,RUSH_ATT,RUSH_YDS,RUSH_TD,FL
0,2020,1,1,Russell Wilson,QB,DEN,1,31,35,322,4,0,3,29,0,0
1,2020,1,2,Aaron Rodgers,QB,GB,1,32,44,364,4,0,1,2,0,0
2,2020,1,3,Josh Allen,QB,BUF,1,33,46,312,2,0,14,57,1,2
3,2020,1,4,Lamar Jackson,QB,BAL,1,20,25,275,3,0,7,45,0,0
4,2020,1,5,Kyler Murray,QB,ARI,1,26,40,230,1,1,13,91,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186,2020,9,34,Mason Rudolph,QB,PIT,1,2,3,3,0,0,0,0,0,0
2239,2020,9,87,Tyrod Taylor,QB,NYG,1,0,0,0,0,0,0,0,0,0
2259,2020,9,107,Jacoby Brissett,QB,CLE,1,0,1,0,0,0,0,0,0,0
2283,2020,9,131,Tim Boyle,QB,DET,1,0,0,0,0,0,1,-1,0,0


In [251]:
#RB - 2020
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2020/RB')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
RB_2020 = pd.DataFrame()
for f in files_csv:
    data = pd.read_csv(f)
    WEEK = f[:-4]
    WEEK = WEEK[12:]
    WEEK = int(WEEK)
    data['WEEK'] = WEEK
    data['WEEK'] = WEEK
    RB_2020 = RB_2020.append(data, ignore_index=True)
RB_2020.fillna(0, inplace=True)
RB_2020 = RB_2020.drop(columns=['Y/A','LG','20+','Y/R','FPTS','FPTS/G','ROST'])
RB_2020 = RB_2020.rename(columns={'ATT':'RUSH_ATT','YDS':'RUSH_YDS','TD':'RUSH_TD','YDS.1':'REC_YDS','TD.1':'REC_TD','Rank':'RANK','Player':'PLAYER'})
RB_2020['YEAR'] = 2020
pd.set_option('precision', 0)
RB_2020[['PLAYER','TEAM']] = RB_2020['PLAYER'].str.split('(', expand=True)
RB_2020['TEAM'] = RB_2020['TEAM'].str.replace(r')','')
RB_2020['POSITION'] = 'RB'
cols = RB_2020.columns.tolist()
cols = cols[12:13] + cols[11:12] + cols[:2] + cols[-1:] + cols[-2:-1] + cols[-5:-4] + cols[2:10]
RB_2020 = RB_2020[cols]
RB_2020 = RB_2020[RB_2020['PLAYER'].notna()]
RB_2020 = RB_2020[RB_2020['G'] == 1]
RB_2020

C:\Users\marcd\AppData\Local\Temp/ipykernel_23064/1858220578.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  RB_2020['TEAM'] = RB_2020['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,G,RUSH_ATT,RUSH_YDS,RUSH_TD,REC,TGT,REC_YDS,REC_TD,FL
0,2020,1,1,Josh Jacobs,RB,LV,1,25,93,3,4,6,46,0,0
1,2020,1,2,Christian McCaffrey,RB,CAR,1,23,97,2,3,4,38,0,0
2,2020,1,3,Ezekiel Elliott,RB,DAL,1,22,96,1,3,4,31,1,0
3,2020,1,4,Malcolm Brown,RB,FA,1,18,79,2,3,4,31,0,0
4,2020,1,5,Nyheim Hines,RB,IND,1,7,28,1,8,8,45,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4779,2020,9,262,Jason Cabinda,RB,DET,1,0,0,0,1,1,4,0,0
4781,2020,9,264,Trenton Cannon,RB,TEN,1,0,0,0,0,0,0,0,0
4786,2020,9,269,Justin Jackson,RB,FA,1,0,0,0,0,0,0,0,0
4787,2020,9,270,Alex Armah Jr.,RB,WAS,1,0,0,0,0,0,0,0,0


In [252]:
#WR - 2020
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2020/WR')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
WR_2020 = pd.DataFrame()
for f in files_csv:
    data = pd.read_csv(f)
    WEEK = f[:-4]
    WEEK = WEEK[12:]
    WEEK = int(WEEK)
    data['WEEK'] = WEEK
    data['WEEK'] = WEEK
    WR_2020 = WR_2020.append(data, ignore_index=True)
WR_2020.fillna(0, inplace=True)
WR_2020 = WR_2020.drop(columns=['LG','20+','Y/R','FPTS','FPTS/G','ROST'])
WR_2020 = WR_2020.rename(columns={'YDS':'REC_YDS','TD':'REC_TD','ATT':'RUSH_ATT','YDS.1':'RUSH_YDS','TD.1':'RUSH_TD','Rank':'RANK','Player':'PLAYER'})
WR_2020['YEAR'] = 2020
pd.set_option('precision', 0)
WR_2020[['PLAYER','TEAM']] = WR_2020['PLAYER'].str.split('(', expand=True)
WR_2020['TEAM'] = WR_2020['TEAM'].str.replace(r')','')
WR_2020['POSITION'] = 'WR'
cols = WR_2020.columns.tolist()
cols = cols[12:13] + cols[11:12] + cols[:2] + cols[-1:] + cols[-2:-1] + cols[-5:-4] + cols[2:10]
WR_2020 = WR_2020[cols]
WR_2020 = WR_2020[WR_2020['PLAYER'].notna()]
WR_2020 = WR_2020[WR_2020['G'] == 1]
WR_2020

C:\Users\marcd\AppData\Local\Temp/ipykernel_23064/1110537024.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  WR_2020['TEAM'] = WR_2020['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,G,REC,TGT,REC_YDS,REC_TD,RUSH_ATT,RUSH_YDS,RUSH_TD,FL
0,2020,1,1,Davante Adams,WR,LV,1,14,17,156,2,0,0,0,0
1,2020,1,2,Calvin Ridley,WR,ATL,1,9,12,130,2,1,-1,0,0
2,2020,1,3,Adam Thielen,WR,MIN,1,6,8,110,2,0,0,0,0
3,2020,1,4,Darius Slayton,WR,NYG,1,6,9,102,2,0,0,0,0
4,2020,1,5,Robbie Anderson,WR,CAR,1,6,8,114,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7516,2020,9,429,Kevin White,WR,NO,1,0,0,0,0,0,0,0,0
7519,2020,9,432,Jake Kumerow,WR,FA,1,0,0,0,0,0,0,0,0
7523,2020,9,436,DeAndre Carter,WR,LAC,1,0,0,0,0,0,0,0,0
7524,2020,9,437,Devin Duvernay,WR,BAL,1,0,0,0,0,1,-1,0,0


In [253]:
#TE - 2020
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2020/TE')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
TE_2020 = pd.DataFrame()
for f in files_csv:
    data = pd.read_csv(f)
    WEEK = f[:-4]
    WEEK = WEEK[12:]
    WEEK = int(WEEK)
    data['WEEK'] = WEEK
    data['WEEK'] = WEEK
    TE_2020 = TE_2020.append(data, ignore_index=True)
TE_2020.fillna(0, inplace=True)
TE_2020 = TE_2020.drop(columns=['LG','20+','Y/R','FPTS','FPTS/G','ROST'])
TE_2020 = TE_2020.rename(columns={'YDS':'REC_YDS','TD':'REC_TD','ATT':'RUSH_ATT','YDS.1':'RUSH_YDS','TD.1':'RUSH_TD','Rank':'RANK','Player':'PLAYER'})
TE_2020['YEAR'] = 2020
pd.set_option('precision', 0)
TE_2020[['PLAYER','TEAM']] = TE_2020['PLAYER'].str.split('(', expand=True)
TE_2020['TEAM'] = TE_2020['TEAM'].str.replace(r')','')
TE_2020['POSITION'] = 'TE'
cols = TE_2020.columns.tolist()
cols = cols[12:13] + cols[11:12] + cols[:2] + cols[-1:] + cols[-2:-1] + cols[-5:-4] + cols[2:10]
TE_2020 = TE_2020[cols]
TE_2020 = TE_2020[TE_2020['PLAYER'].notna()]
TE_2020 = TE_2020[TE_2020['G'] == 1]
TE_2020

C:\Users\marcd\AppData\Local\Temp/ipykernel_23064/3340007095.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  TE_2020['TEAM'] = TE_2020['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,G,REC,TGT,REC_YDS,REC_TD,RUSH_ATT,RUSH_YDS,RUSH_TD,FL
0,2020,1,1,Mark Andrews,TE,BAL,1,5,6,58,2,0,0,0,0
1,2020,1,2,Dallas Goedert,TE,PHI,1,8,9,101,1,0,0,0,0
2,2020,1,3,Noah Fant,TE,SEA,1,5,6,81,1,0,0,0,0
3,2020,1,4,T.J. Hockenson,TE,DET,1,5,5,56,1,0,0,0,0
4,2020,1,5,Travis Kelce,TE,KC,1,6,6,50,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4571,2020,9,260,Ross Travis,TE,FA,1,0,0,0,0,0,0,0,0
4572,2020,9,261,Cameron Brate,TE,TB,1,0,0,0,0,0,0,0,0
4574,2020,9,263,Daniel Brown,TE,FA,1,0,0,0,0,0,0,0,0
4576,2020,9,265,MyCole Pruitt,TE,FA,1,0,0,0,0,0,0,0,0


#### Merge All 2020 Weekly Data Together

In [254]:
weekly_2020 = QB_2020.append([RB_2020, WR_2020, TE_2020])
weekly_2020.fillna(0, inplace=True)

RR_TD = 6     # Rushing/Receiving TDs
RR_YDS = 0.1  # Rushing/Receiving Yards
REC = 1     # Receptions
FL = -2       # Fumble Loses
P_TD = 4      # Passing TDs
P_YDS = 0.04  # Passing Yards
INT = -2      # Interceptions

weekly_2020['FPTS'] = RR_TD*(weekly_2020['RUSH_TD'] + weekly_2020['REC_TD']) + RR_YDS*(weekly_2020['RUSH_YDS'] + weekly_2020['REC_YDS']) + REC*(weekly_2020['REC']) + \
FL*(weekly_2020['FL']) + P_TD*(weekly_2020['PASS_TD']) + P_YDS*(weekly_2020['PASS_YDS']) + INT*(weekly_2020['PASS_INT'])

weekly_2020 = weekly_2020.round({'FPTS': 2})
weekly_2020

weekly_2020

,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,G,PASS_CMP,PASS_ATT,PASS_YDS,...,PASS_INT,RUSH_ATT,RUSH_YDS,RUSH_TD,FL,REC,TGT,REC_YDS,REC_TD,FPTS
0,2020,1,1,Russell Wilson,QB,DEN,1,31,35,322,...,0,3,29,0,0,0,0,0,0,32
1,2020,1,2,Aaron Rodgers,QB,GB,1,32,44,364,...,0,1,2,0,0,0,0,0,0,31
2,2020,1,3,Josh Allen,QB,BUF,1,33,46,312,...,0,14,57,1,2,0,0,0,0,28
3,2020,1,4,Lamar Jackson,QB,BAL,1,20,25,275,...,0,7,45,0,0,0,0,0,0,28
4,2020,1,5,Kyler Murray,QB,ARI,1,26,40,230,...,1,13,91,1,0,0,0,0,0,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4571,2020,9,260,Ross Travis,TE,FA,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4572,2020,9,261,Cameron Brate,TE,TB,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4574,2020,9,263,Daniel Brown,TE,FA,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4576,2020,9,265,MyCole Pruitt,TE,FA,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Add 2020 Schedule / Opponent column

In [255]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Schedule')
schedule_2020 = pd.read_csv('2020_Schedule.csv')
schedule_2020 = schedule_2020.drop(['W18'], axis=1)
schedule_2020 = schedule_2020.rename(columns={'Team':'TEAM'})
schedule_2020 = schedule_2020.melt(id_vars=['TEAM'], var_name='WEEK')
schedule_2020['WEEK'] = schedule_2020['WEEK'].map(lambda x: x.lstrip('W'))
schedule_2020['value'] = schedule_2020['value'].map(lambda x: x.lstrip('@'))
schedule_2020 = schedule_2020.astype({'WEEK': int})
schedule_2020

,TEAM,WEEK,value
0,ARI,1,SF
1,ATL,1,SEA
2,BAL,1,CLE
3,BUF,1,NYJ
4,CAR,1,LV
...,...,...,...
539,SEA,17,SF
540,SF,17,SEA
541,TB,17,ATL
542,TEN,17,HOU


In [256]:
weekly_2020 = pd.merge(weekly_2020, schedule_2020, on=['TEAM','WEEK'])
weekly_2020 = weekly_2020.rename(columns={'value':'OPP'})
cols = weekly_2020.columns.tolist()
cols = cols[0:6] + cols[-1:] + cols[6:15] + cols[16:20] + cols[15:16] + cols[20:21]
weekly_2020 = weekly_2020[cols]
weekly_2020

,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,OPP,G,PASS_CMP,PASS_ATT,...,PASS_INT,RUSH_ATT,RUSH_YDS,RUSH_TD,REC,TGT,REC_YDS,REC_TD,FL,FPTS
0,2020,1,1,Russell Wilson,QB,DEN,TEN,1,31,35,...,0,3,29,0,0,0,0,0,0,32
1,2020,1,16,Melvin Gordon III,RB,DEN,TEN,1,0,0,...,0,15,78,1,3,3,8,0,1,16
2,2020,1,123,Mike Boone,RB,DEN,TEN,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020,1,144,Andrew Beck,RB,DEN,TEN,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020,1,46,Jerry Jeudy,WR,DEN,TEN,1,0,0,...,0,0,0,0,4,8,56,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5267,2020,9,50,Zach Pascal,WR,PHI,BYE,1,0,0,...,0,0,0,0,5,6,55,0,0,10
5268,2020,9,97,Trent Taylor,WR,CIN,BYE,1,0,0,...,0,0,0,0,1,4,9,0,0,2
5269,2020,9,9,Hayden Hurst,TE,CIN,BYE,1,0,0,...,0,0,0,0,7,8,62,0,0,13
5270,2020,7,21,Johnny Mundt,TE,MIN,BYE,1,0,0,...,0,0,0,0,3,3,48,0,0,8


#### Add Opponent Position Defensive Rankings and Remove Irrelevant Players and Add FPTS/G

In [257]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Defense_Rankings')
defense_2020 = pd.read_csv('Defense_Rankings_2020.csv')
defense_2020 = defense_2020.melt(id_vars=['OPP'], var_name='RANK')
defense_2020 = defense_2020.rename(columns={'RANK':'POSITION','value':'RANK'})

weekly_2020 = pd.merge(weekly_2020, defense_2020, on=['OPP','POSITION'])
G_SUM_2020 = weekly_2020.groupby('PLAYER')['G'].sum()
weekly_2020 = pd.merge(weekly_2020, G_SUM_2020, on='PLAYER')
weekly_2020 = weekly_2020.rename(columns={'RANK_x':'RANK','RANK_y':'OPP_DEF_POS_RANK','G_y':'G_SUM'})
weekly_2020 = weekly_2020.drop(['G_x'], axis=1)
cols = weekly_2020.columns.tolist()
cols = cols[0:7] + cols[-2:-1] + cols[-1:] + cols[7:21]
weekly_2020 = weekly_2020[cols]

weekly_2020 = weekly_2020.astype({'PASS_CMP':int,'PASS_ATT':int,'PASS_YDS':int,'PASS_TD':int,'PASS_INT':int,'RUSH_ATT':int,'RUSH_YDS':int,'RUSH_TD':int,'FL':int,'REC':int,'TGT':int,'REC_YDS':int,'REC_TD':int,'FPTS':float})

weekly_2020_drop = weekly_2020[(weekly_2020.PASS_CMP == 0) & (weekly_2020.PASS_ATT == 0) & (weekly_2020.PASS_YDS == 0) & (weekly_2020.PASS_TD == 0) & (weekly_2020.PASS_INT == 0) & (weekly_2020.RUSH_ATT == 0) & (weekly_2020.RUSH_YDS == 0) & (weekly_2020.RUSH_TD == 0) & \
(weekly_2020.FL == 0) & (weekly_2020.REC == 0) & (weekly_2020.TGT == 0) & (weekly_2020.REC_YDS == 0) & (weekly_2020.REC_TD == 0) & (weekly_2020.FPTS == 0)].index

weekly_2020.drop(weekly_2020_drop, inplace=True)

weekly_2020['FPTS/G'] = weekly_2020.groupby(['PLAYER'])['FPTS'].transform('mean')
weekly_2020['FPTS/G'] = weekly_2020['FPTS/G'].apply(lambda x: round(x, 2))

weekly_2020['FPTS_DIFF'] = weekly_2020['FPTS'] - weekly_2020['FPTS/G']
weekly_2020['FPTS_DIFF'] = weekly_2020['FPTS_DIFF'].apply(lambda x: round(x, 2))

weekly_2020['PLAYER'] = weekly_2020['PLAYER'].map(lambda x: x.rstrip(' '))

weekly_2020

,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,OPP,OPP_DEF_POS_RANK,G_SUM,PASS_CMP,...,RUSH_YDS,RUSH_TD,REC,TGT,REC_YDS,REC_TD,FL,FPTS,FPTS/G,FPTS_DIFF
0,2020,1,1,Russell Wilson,QB,DEN,TEN,28,15,31,...,29,0,0,0,0,0,0,32,22,10
1,2020,4,13,Russell Wilson,QB,DEN,NYJ,30,15,24,...,5,0,0,0,0,0,0,21,22,-1
2,2020,12,19,Russell Wilson,QB,DEN,NO,5,15,22,...,12,0,0,0,0,0,0,14,22,-8
3,2020,2,4,Russell Wilson,QB,DEN,PIT,2,15,21,...,39,0,0,0,0,0,0,33,22,11
4,2020,16,15,Russell Wilson,QB,DEN,LAC,21,15,20,...,9,1,0,0,0,0,0,20,22,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5133,2020,15,8,Marcus Mariota,QB,ATL,TB,16,1,17,...,88,1,0,0,0,0,0,26,26,0
5134,2020,16,36,David Blough,QB,DET,TB,16,1,6,...,18,0,0,0,0,0,0,2,2,0
5136,2020,17,135,Nate Sudfeld,QB,SF,SEA,26,1,5,...,12,0,0,0,0,0,1,-2,-2,0
5137,2020,4,32,Brian Hoyer,QB,NE,KC,23,1,15,...,8,0,0,0,0,0,1,2,2,0


#### Save Weekly 2020 Data

In [258]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football')
writer = pd.ExcelWriter('weekly_2020.xlsx')
weekly_2020.to_excel(writer,'sheet1')
writer.save()

#### Get 2021 Weekly Data

In [259]:
#QB - 2021
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2021/QB')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
QB_2021 = pd.DataFrame()
for f in files_csv:
    data = pd.read_csv(f)
    WEEK = f[:-4]
    WEEK = WEEK[12:]
    WEEK = int(WEEK)
    data['WEEK'] = WEEK
    data['WEEK'] = WEEK
    QB_2021 = QB_2021.append(data, ignore_index=True)
QB_2021.fillna(0, inplace=True)
QB_2021 = QB_2021.drop(columns=['Y/A','SACKS','PCT','FPTS','FPTS/G','ROST'])
QB_2021 = QB_2021.rename(columns={'CMP':'PASS_CMP','ATT':'PASS_ATT','YDS':'PASS_YDS','TD':'PASS_TD','INT':'PASS_INT','ATT.1':'RUSH_ATT','YDS.1':'RUSH_YDS','TD.1':'RUSH_TD','Rank':'RANK','Player':'PLAYER'})
QB_2021['YEAR'] = 2021
pd.set_option('precision', 0)
QB_2021[['PLAYER','TEAM']] = QB_2021['PLAYER'].str.split('(', expand=True)
QB_2021['TEAM'] = QB_2021['TEAM'].str.replace(r')','')
QB_2021['POSITION'] = 'QB'
cols = QB_2021.columns.tolist()
cols = cols[13:14] + cols[12:13] + cols[0:2] + cols[-1:] + cols[-2:-1] + cols[-5:-4] + cols[2:11]
QB_2021 = QB_2021[cols]
QB_2021 = QB_2021[QB_2021['PLAYER'].notna()]
QB_2021 = QB_2021[QB_2021['G'] == 1]
QB_2021

C:\Users\marcd\AppData\Local\Temp/ipykernel_23064/2589213125.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  QB_2021['TEAM'] = QB_2021['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,G,PASS_CMP,PASS_ATT,PASS_YDS,PASS_TD,PASS_INT,RUSH_ATT,RUSH_YDS,RUSH_TD,FL
0,2021,1,1,Kyler Murray,QB,ARI,1,21,32,289,4,1,5,20,1,0
1,2021,1,2,Patrick Mahomes II,QB,KC,1,27,36,337,3,0,5,18,1,0
2,2021,1,3,Jared Goff,QB,DET,1,38,57,338,3,1,3,14,0,0
3,2021,1,4,Jameis Winston,QB,NO,1,14,20,148,5,0,6,37,0,0
4,2021,1,5,Tom Brady,QB,TB,1,32,50,379,4,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2248,2021,9,34,Chris Streveler,QB,FA,1,0,0,0,0,0,1,2,0,0
2249,2021,9,35,Brandon Allen,QB,CIN,1,1,2,6,0,0,1,0,0,0
2250,2021,9,36,Sam Ehlinger,QB,IND,1,0,0,0,0,0,1,1,0,0
2341,2021,9,127,Logan Woodside,QB,TEN,1,0,0,0,0,0,1,-1,0,0


In [260]:
#RB - 2021
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2021/RB')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
RB_2021 = pd.DataFrame()
for f in files_csv:
    data = pd.read_csv(f)
    WEEK = f[:-4]
    WEEK = WEEK[12:]
    WEEK = int(WEEK)
    data['WEEK'] = WEEK
    data['WEEK'] = WEEK
    RB_2021 = RB_2021.append(data, ignore_index=True)
RB_2021.fillna(0, inplace=True)
RB_2021 = RB_2021.drop(columns=['Y/A','LG','20+','Y/R','FPTS','FPTS/G','ROST'])
RB_2021 = RB_2021.rename(columns={'ATT':'RUSH_ATT','YDS':'RUSH_YDS','TD':'RUSH_TD','YDS.1':'REC_YDS','TD.1':'REC_TD','Rank':'RANK','Player':'PLAYER'})
RB_2021['YEAR'] = 2021
pd.set_option('precision', 0)
RB_2021[['PLAYER','TEAM']] = RB_2021['PLAYER'].str.split('(', expand=True)
RB_2021['TEAM'] = RB_2021['TEAM'].str.replace(r')','')
RB_2021['POSITION'] = 'RB'
cols = RB_2021.columns.tolist()
cols = cols[12:13] + cols[11:12] + cols[:2] + cols[-1:] + cols[-2:-1] + cols[-5:-4] + cols[2:10]
RB_2021 = RB_2021[cols]
RB_2021 = RB_2021[RB_2021['PLAYER'].notna()]
RB_2021 = RB_2021[RB_2021['G'] == 1]
RB_2021

C:\Users\marcd\AppData\Local\Temp/ipykernel_23064/2211467662.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  RB_2021['TEAM'] = RB_2021['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,G,RUSH_ATT,RUSH_YDS,RUSH_TD,REC,TGT,REC_YDS,REC_TD,FL
0,2021,1,1,Christian McCaffrey,RB,CAR,1,21,98,0,9,9,89,0,0
1,2021,1,2,Joe Mixon,RB,CIN,1,29,127,1,4,4,23,0,0
2,2021,1,3,Nick Chubb,RB,CLE,1,15,83,2,2,2,18,0,1
3,2021,1,4,Jamaal Williams,RB,DET,1,9,54,1,8,9,56,0,0
4,2021,1,5,D'Andre Swift,RB,DET,1,11,39,0,8,11,65,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4727,2021,9,231,Dwayne Washington,RB,NO,1,0,0,0,0,0,0,0,0
4728,2021,9,232,Andy Janovich,RB,HOU,1,0,0,0,0,0,0,0,0
4736,2021,9,240,Trey Edmunds,RB,PIT,1,0,0,0,0,0,0,0,0
4738,2021,9,242,Trenton Cannon,RB,TEN,1,0,0,0,0,0,0,0,0


In [261]:
#WR - 2021
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2021/WR')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
WR_2021 = pd.DataFrame()
for f in files_csv:
    data = pd.read_csv(f)
    WEEK = f[:-4]
    WEEK = WEEK[12:]
    WEEK = int(WEEK)
    data['WEEK'] = WEEK
    data['WEEK'] = WEEK
    WR_2021 = WR_2021.append(data, ignore_index=True)
WR_2021.fillna(0, inplace=True)
WR_2021 = WR_2021.drop(columns=['LG','20+','Y/R','FPTS','FPTS/G','ROST'])
WR_2021 = WR_2021.rename(columns={'YDS':'REC_YDS','TD':'REC_TD','ATT':'RUSH_ATT','YDS.1':'RUSH_YDS','TD.1':'RUSH_TD','Rank':'RANK','Player':'PLAYER'})
WR_2021['YEAR'] = 2021
pd.set_option('precision', 0)
WR_2021[['PLAYER','TEAM']] = WR_2021['PLAYER'].str.split('(', expand=True)
WR_2021['TEAM'] = WR_2021['TEAM'].str.replace(r')','')
WR_2021['POSITION'] = 'WR'
cols = WR_2021.columns.tolist()
cols = cols[12:13] + cols[11:12] + cols[:2] + cols[-1:] + cols[-2:-1] + cols[-5:-4] + cols[2:10]
WR_2021 = WR_2021[cols]
WR_2021 = WR_2021[WR_2021['PLAYER'].notna()]
WR_2021 = WR_2021[WR_2021['G'] == 1]
WR_2021

C:\Users\marcd\AppData\Local\Temp/ipykernel_23064/27969831.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  WR_2021['TEAM'] = WR_2021['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,G,REC,TGT,REC_YDS,REC_TD,RUSH_ATT,RUSH_YDS,RUSH_TD,FL
0,2021,1,1,Amari Cooper,WR,CLE,1,13,17,139,2,0,0,0,0
1,2021,1,2,Tyreek Hill,WR,MIA,1,11,15,197,1,1,4,0,0
2,2021,1,3,Deebo Samuel,WR,SF,1,9,12,189,1,0,0,0,1
3,2021,1,4,Adam Thielen,WR,MIN,1,9,10,92,2,0,0,0,0
4,2021,1,5,Corey Davis,WR,NYJ,1,5,7,97,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7780,2021,9,423,Jalen Reagor,WR,PHI,1,1,1,-6,0,0,0,0,0
7781,2021,9,424,Ray-Ray McCloud,WR,SF,1,1,1,12,0,0,0,0,1
7782,2021,9,425,Marcus Johnson,WR,SF,1,0,0,0,0,1,-7,0,0
7783,2021,9,426,Jakeem Grant Sr.,WR,CLE,1,0,0,0,0,0,0,0,1


In [262]:
#TE - 2021
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2021/TE')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
TE_2021 = pd.DataFrame()
for f in files_csv:
    data = pd.read_csv(f)
    WEEK = f[:-4]
    WEEK = WEEK[12:]
    WEEK = int(WEEK)
    data['WEEK'] = WEEK
    data['WEEK'] = WEEK
    TE_2021 = TE_2021.append(data, ignore_index=True)
TE_2021.fillna(0, inplace=True)
TE_2021 = TE_2021.drop(columns=['LG','20+','Y/R','FPTS','FPTS/G','ROST'])
TE_2021 = TE_2021.rename(columns={'YDS':'REC_YDS','TD':'REC_TD','ATT':'RUSH_ATT','YDS.1':'RUSH_YDS','TD.1':'RUSH_TD','Rank':'RANK','Player':'PLAYER'})
TE_2021['YEAR'] = 2021
pd.set_option('precision', 0)
TE_2021[['PLAYER','TEAM']] = TE_2021['PLAYER'].str.split('(', expand=True)
TE_2021['TEAM'] = TE_2021['TEAM'].str.replace(r')','')
TE_2021['POSITION'] = 'TE'
cols = TE_2021.columns.tolist()
cols = cols[12:13] + cols[11:12] + cols[:2] + cols[-1:] + cols[-2:-1] + cols[-5:-4] + cols[2:10]
TE_2021 = TE_2021[cols]
TE_2021 = TE_2021[TE_2021['PLAYER'].notna()]
TE_2021 = TE_2021[TE_2021['G'] == 1]
TE_2021

C:\Users\marcd\AppData\Local\Temp/ipykernel_23064/1684915415.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  TE_2021['TEAM'] = TE_2021['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,G,REC,TGT,REC_YDS,REC_TD,RUSH_ATT,RUSH_YDS,RUSH_TD,FL
0,2021,1,1,Rob Gronkowski,TE,FA,1,8,8,90,2,0,0,0,0
1,2021,1,2,Travis Kelce,TE,KC,1,6,7,76,2,0,0,0,0
2,2021,1,3,T.J. Hockenson,TE,DET,1,8,11,97,1,0,0,0,0
3,2021,1,4,Darren Waller,TE,LV,1,10,19,105,1,0,0,0,0
4,2021,1,5,Juwan Johnson,TE,NO,1,3,3,21,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4443,2021,9,225,Colin Thompson,TE,CAR,1,0,0,0,0,0,0,0,0
4451,2021,9,233,Cethan Carter,TE,MIA,1,0,0,0,0,0,0,0,0
4460,2021,9,242,MyCole Pruitt,TE,FA,1,0,0,0,0,0,0,0,0
4461,2021,9,243,Blake Bell,TE,KC,1,0,1,0,0,0,0,0,0


#### Merge All 2021 Weekly Data Together

In [263]:
weekly_2021 = QB_2021.append([RB_2021, WR_2021, TE_2021])
weekly_2021.fillna(0, inplace=True)

RR_TD = 6     # Rushing/Receiving TDs
RR_YDS = 0.1  # Rushing/Receiving Yards
REC = 1     # Receptions
FL = -2       # Fumble Loses
P_TD = 4      # Passing TDs
P_YDS = 0.04  # Passing Yards
INT = -2      # Interceptions

weekly_2021['FPTS'] = RR_TD*(weekly_2021['RUSH_TD'] + weekly_2021['REC_TD']) + RR_YDS*(weekly_2021['RUSH_YDS'] + weekly_2021['REC_YDS']) + REC*(weekly_2021['REC']) + \
FL*(weekly_2021['FL']) + P_TD*(weekly_2021['PASS_TD']) + P_YDS*(weekly_2021['PASS_YDS']) + INT*(weekly_2021['PASS_INT'])

weekly_2021 = weekly_2021.round({'FPTS': 2})
weekly_2021

weekly_2021[weekly_2021['PLAYER'] == 'Jonathan Taylor ']

,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,G,PASS_CMP,PASS_ATT,PASS_YDS,...,PASS_INT,RUSH_ATT,RUSH_YDS,RUSH_TD,FL,REC,TGT,REC_YDS,REC_TD,FPTS
15,2021,1,16,Jonathan Taylor,RB,IND,1,0,0,0,...,0,17,56,0,0,6,7,60,0,18
267,2021,10,4,Jonathan Taylor,RB,IND,1,0,0,0,...,0,21,116,1,0,6,8,10,0,25
529,2021,11,1,Jonathan Taylor,RB,IND,1,0,0,0,...,0,32,185,4,0,3,3,19,1,53
807,2021,12,12,Jonathan Taylor,RB,IND,1,0,0,0,...,0,16,83,1,0,4,5,14,0,20
1061,2021,13,2,Jonathan Taylor,RB,IND,1,0,0,0,...,0,32,143,2,1,0,0,0,0,24
1588,2021,15,2,Jonathan Taylor,RB,IND,1,0,0,0,...,0,29,170,1,0,0,0,0,0,23
1874,2021,16,23,Jonathan Taylor,RB,IND,1,0,0,0,...,0,27,108,0,0,0,2,0,0,11
2130,2021,17,14,Jonathan Taylor,RB,IND,1,0,0,0,...,0,20,108,1,0,1,2,6,0,18
2405,2021,18,23,Jonathan Taylor,RB,IND,1,0,0,0,...,0,15,77,0,0,3,4,18,0,12
2690,2021,2,43,Jonathan Taylor,RB,IND,1,0,0,0,...,0,15,51,0,0,1,1,2,0,6


#### Add 2021 Schedule / Opponent column

In [264]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Schedule')
schedule_2021 = pd.read_csv('2021_Schedule.csv')
schedule_2021 = schedule_2021.rename(columns={'Team':'TEAM'})
schedule_2021 = schedule_2021.melt(id_vars=['TEAM'], var_name='WEEK')
schedule_2021['WEEK'] = schedule_2021['WEEK'].map(lambda x: x.lstrip('W'))
schedule_2021['value'] = schedule_2021['value'].map(lambda x: x.lstrip('@'))
schedule_2021 = schedule_2021.astype({'WEEK': int})
schedule_2021

,TEAM,WEEK,value
0,ARI,1,TEN
1,ATL,1,PHI
2,BAL,1,LV
3,BUF,1,PIT
4,CAR,1,NYJ
...,...,...,...
571,SEA,18,ARI
572,SF,18,LAR
573,TB,18,CAR
574,TEN,18,HOU


In [265]:
weekly_2021 = pd.merge(weekly_2021, schedule_2021, on=['TEAM','WEEK'])
weekly_2021 = weekly_2021.rename(columns={'value':'OPP'})
cols = weekly_2021.columns.tolist()
cols = cols[0:6] + cols[-1:] + cols[6:15] + cols[16:20] + cols[15:16] + cols[20:21]
weekly_2021 = weekly_2021[cols]
weekly_2021

,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,OPP,G,PASS_CMP,PASS_ATT,...,PASS_INT,RUSH_ATT,RUSH_YDS,RUSH_TD,REC,TGT,REC_YDS,REC_TD,FL,FPTS
0,2021,1,1,Kyler Murray,QB,ARI,TEN,1,21,32,...,1,5,20,1,0,0,0,0,0,3e+01
1,2021,1,44,James Conner,RB,ARI,TEN,1,0,0,...,0,16,53,0,0,0,0,0,0,5e+00
2,2021,1,80,Darrel Williams,RB,ARI,TEN,1,0,0,...,0,1,4,0,0,0,0,0,0,4e-01
3,2021,1,92,Jonathan Ward,RB,ARI,TEN,1,0,0,...,0,0,0,0,0,0,0,0,0,0e+00
4,2021,1,7,DeAndre Hopkins,WR,ARI,TEN,1,0,0,...,0,0,0,0,6,8,83,2,0,3e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5316,2021,7,67,Gunner Olszewski,WR,PIT,BYE,1,0,0,...,0,0,0,0,1,1,22,0,0,3e+00
5317,2021,9,22,Russell Gage,WR,TB,BYE,1,0,0,...,0,0,0,0,7,8,64,0,0,1e+01
5318,2021,9,51,Marquise Goodwin,WR,SEA,BYE,1,0,0,...,0,0,0,0,1,4,50,0,0,6e+00
5319,2021,6,14,C.J. Uzomah,TE,NYJ,BYE,1,0,0,...,0,0,0,0,3,3,15,1,0,1e+01


#### Add Opponent Position Defensive Rankings and Remove Irrelevant Players and Add FPTS/G

In [266]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Defense_Rankings')
defense_2021 = pd.read_csv('Defense_Rankings_2021.csv')
defense_2021 = defense_2021.melt(id_vars=['OPP'], var_name='RANK')
defense_2021 = defense_2021.rename(columns={'RANK':'POSITION','value':'RANK'})

weekly_2021 = pd.merge(weekly_2021, defense_2021, on=['OPP','POSITION'])
G_SUM_2021 = weekly_2021.groupby('PLAYER')['G'].sum()
weekly_2021 = pd.merge(weekly_2021, G_SUM_2021, on='PLAYER')
weekly_2021 = weekly_2021.rename(columns={'RANK_x':'RANK','RANK_y':'OPP_DEF_POS_RANK','G_y':'G_SUM'})
weekly_2021 = weekly_2021.drop(['G_x'], axis=1)
cols = weekly_2021.columns.tolist()
cols = cols[0:7] + cols[-2:-1] + cols[-1:] + cols[7:21]
weekly_2021 = weekly_2021[cols]

weekly_2021 = weekly_2021.astype({'PASS_CMP':int,'PASS_ATT':int,'PASS_YDS':int,'PASS_TD':int,'PASS_INT':int,'RUSH_ATT':int,'RUSH_YDS':int,'RUSH_TD':int,'FL':int,'REC':int,'TGT':int,'REC_YDS':int,'REC_TD':int,'FPTS':float})

weekly_2021_drop = weekly_2021[(weekly_2021.PASS_CMP == 0) & (weekly_2021.PASS_ATT == 0) & (weekly_2021.PASS_YDS == 0) & (weekly_2021.PASS_TD == 0) & (weekly_2021.PASS_INT == 0) & (weekly_2021.RUSH_ATT == 0) & (weekly_2021.RUSH_YDS == 0) & (weekly_2021.RUSH_TD == 0) & \
(weekly_2021.FL == 0) & (weekly_2021.REC == 0) & (weekly_2021.TGT == 0) & (weekly_2021.REC_YDS == 0) & (weekly_2021.REC_TD == 0) & (weekly_2021.FPTS == 0)].index

weekly_2021.drop(weekly_2021_drop, inplace=True)

weekly_2021['FPTS/G'] = weekly_2021.groupby(['PLAYER'])['FPTS'].transform('mean')
weekly_2021['FPTS/G'] = weekly_2021['FPTS/G'].apply(lambda x: round(x, 2))

weekly_2021['FPTS_DIFF'] = weekly_2021['FPTS'] - weekly_2021['FPTS/G']
weekly_2021['FPTS_DIFF'] = weekly_2021['FPTS_DIFF'].apply(lambda x: round(x, 2))

weekly_2021['PLAYER'] = weekly_2021['PLAYER'].map(lambda x: x.rstrip(' '))

weekly_2021

,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,OPP,OPP_DEF_POS_RANK,G_SUM,PASS_CMP,...,RUSH_YDS,RUSH_TD,REC,TGT,REC_YDS,REC_TD,FL,FPTS,FPTS/G,FPTS_DIFF
0,2021,1,1,Kyler Murray,QB,ARI,TEN,20,14,21,...,20,1,0,0,0,0,0,34,21,12
1,2021,6,6,Kyler Murray,QB,ARI,CLE,12,14,20,...,6,0,0,0,0,0,0,26,21,4
2,2021,5,23,Kyler Murray,QB,ARI,SF,19,14,22,...,1,0,0,0,0,0,0,14,21,-8
3,2021,8,26,Kyler Murray,QB,ARI,GB,25,14,22,...,21,0,0,0,0,0,0,9,21,-12
4,2021,17,6,Kyler Murray,QB,ARI,DAL,11,14,26,...,44,0,0,0,0,0,0,23,21,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5257,2021,14,211,Vyncint Smith,WR,TB,BUF,1,1,0,...,0,0,0,1,0,0,0,0,0,0
5258,2021,18,99,Tarik Black,WR,NYJ,BUF,1,1,0,...,0,0,1,2,10,0,0,2,2,0
5259,2021,17,7,Kristian Wilkerson,WR,NE,JAX,26,1,0,...,0,0,4,8,42,2,0,20,20,0
5260,2021,18,34,Richard Rodgers,TE,PHI,DAL,16,1,0,...,0,0,2,2,11,0,0,3,3,0


#### Save Weekly 2021 Data

In [208]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football')
writer = pd.ExcelWriter('weekly_2021.xlsx')
weekly_2021.to_excel(writer,'sheet1')
writer.save()